In [0]:
!pip install pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-cp36-none-any.whl size=26819 sha256=72fe059d76ae58c825585ae03695f097fab18b7148bc388b92af71f1ff61a7d2
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from pandasql import sqldf
from sklearn.ensemble import StackingClassifier

In [47]:
df = pd.read_csv('/content/drive/My Drive/Datasets/flight_delays.csv')
df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [48]:
X, y = df[['Distance', 'DepTime']].values, df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=17)
lr = LogisticRegression( solver='lbfgs',random_state=17)

lr.fit(X_train, y_train)
y_pred = lr.predict_proba(X_test)[:, 1]

print('AUC-ROC:',roc_auc_score(y_test, y_pred))

AUC-ROC: 0.6795697123357751


In [49]:
# df.isna().sum() - нет нанов
# Добавление маршрута сильно увеличивает количество категориальных признаков при ван хот кодировании 
X = df.copy()
y = X.loc[:,'dep_delayed_15min'] = X['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

# Предотвращаем Knowledge leaking
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# попробум построить признак День-Месяц-Вылет(Прилет) со значениями частоты задержки, причем значения расчитываем по трейну
X_train.loc[:,'DateOrigin'] = X_train['Month'] + X_train['DayofMonth'] + X_train['Origin']
X_train.loc[:,'DateDest'] = X_train['Month'] + X_train['DayofMonth'] + X_train['Dest']
X_test.loc[:,'DateOrigin'] = X_test['Month'] + X_test['DayofMonth'] + X_test['Origin']
X_test.loc[:,'DateDest'] = X_test['Month'] + X_test['DayofMonth'] + X_test['Dest']

for encoding_feature in ['DateOrigin', 'DateDest']:
    counts = X_train.groupby(encoding_feature).size()
    y_counts = X_train.groupby(encoding_feature)['dep_delayed_15min'].value_counts()[:,1]

    temp = pd.DataFrame(counts).join(pd.DataFrame(y_counts), on=encoding_feature, how='left').fillna(0)
    temp.columns = ['Count', '1_count']
    temp = pd.DataFrame(temp['1_count'] / temp['Count'])
    temp.applymap(lambda x: np.piecewise(x, [x <= 0, 0 < x < 0.25, 0.25 <= x < 0.5, 0.5 <= x < 0.75, x >= 0.75], [0, 1, 2, 3, 4]))
    temp.columns = [encoding_feature + 'Mean']
    
    X_train = X_train.join(temp, on=encoding_feature, how='left')
    X_test = X_test.join(temp, on=encoding_feature, how='left')
    X_test = X_test.fillna(0)

X_train = X_train.drop(columns=['dep_delayed_15min', 'DateOrigin', 'DateDest'])
X_test = X_test.drop(columns=['dep_delayed_15min', 'DateOrigin', 'DateDest'])

X_dummies = pd.concat([X_train, X_test], sort = False)

n_train = len(X_train)
n_test = len(X_test)

X_dummies = pd.get_dummies(X_dummies)

X_train = X_dummies.head(n_train)
X_test = X_dummies.tail(n_test)
# Пробовал  - качество сильно падает - не используем
X_train = X_train.drop(columns=['DateOriginMean', 'DateDestMean'])
X_test = X_test.drop(columns=['DateOriginMean', 'DateDestMean'])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [0]:
# Долго и кропотливо подбираем параметры
eval_set = [(X_test, y_test)]
xgb = XGBClassifier(
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    learning_rate =0.1,
    n_estimators=50,
    max_depth=5,
    min_child_weight=1,
    scale_pos_weight=1)

xgb.fit(X_train, y_train, eval_metric="auc", eval_set=eval_set, verbose=True, early_stopping_rounds=10)
y_pred = xgb.predict_proba(X_test)[:,1]
print('AUC-ROC:', roc_auc_score(y_test, y_pred))
#pd.DataFrame(zip(xgb.feature_importances_, X.columns)).sort_values(by=0, ascending=False).head(20)

[0]	validation_0-auc:0.690378
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.694522
[2]	validation_0-auc:0.699313
[3]	validation_0-auc:0.701177
[4]	validation_0-auc:0.703585
[5]	validation_0-auc:0.704943
[6]	validation_0-auc:0.707461
[7]	validation_0-auc:0.70877
[8]	validation_0-auc:0.708728
[9]	validation_0-auc:0.708468
[10]	validation_0-auc:0.708023
[11]	validation_0-auc:0.708074
[12]	validation_0-auc:0.707822
[13]	validation_0-auc:0.707663
[14]	validation_0-auc:0.707969
[15]	validation_0-auc:0.708439
[16]	validation_0-auc:0.70867
[17]	validation_0-auc:0.70931
[18]	validation_0-auc:0.709447
[19]	validation_0-auc:0.709879
[20]	validation_0-auc:0.710395
[21]	validation_0-auc:0.710777
[22]	validation_0-auc:0.711288
[23]	validation_0-auc:0.711655
[24]	validation_0-auc:0.711812
[25]	validation_0-auc:0.712049
[26]	validation_0-auc:0.712067
[27]	validation_0-auc:0.712438
[28]	validation_0-auc:0.712701
[29]	validation_0-auc:0.712829
[30]	validation_0-a

In [0]:
xgb_best = xgb

param1 = {
 'max_depth':range(3, 10, 2),
 'min_child_weight':range(1, 6, 2)
}
xgb_grid = GridSearchCV(xgb_best, param1, verbose=True, scoring = 'roc_auc', n_jobs=6, cv = 2)
xgb_grid.fit(X_train, y_train, eval_metric="auc", eval_set=eval_set, verbose=True, early_stopping_rounds=10)
y_pred = xgb_grid.predict_proba(X_test)[:,1]
print('AUC-ROC:', roc_auc_score(y_test, y_pred))

Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  24 out of  24 | elapsed: 15.1min finished


[0]	validation_0-auc:0.698452
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.700652
[2]	validation_0-auc:0.709302
[3]	validation_0-auc:0.711336
[4]	validation_0-auc:0.71286
[5]	validation_0-auc:0.71458
[6]	validation_0-auc:0.716935
[7]	validation_0-auc:0.71772
[8]	validation_0-auc:0.717788
[9]	validation_0-auc:0.718966
[10]	validation_0-auc:0.718969
[11]	validation_0-auc:0.719441
[12]	validation_0-auc:0.71912
[13]	validation_0-auc:0.719242
[14]	validation_0-auc:0.719863
[15]	validation_0-auc:0.720345
[16]	validation_0-auc:0.720482
[17]	validation_0-auc:0.721614
[18]	validation_0-auc:0.721948
[19]	validation_0-auc:0.722556
[20]	validation_0-auc:0.723666
[21]	validation_0-auc:0.72416
[22]	validation_0-auc:0.724587
[23]	validation_0-auc:0.724683
[24]	validation_0-auc:0.725022
[25]	validation_0-auc:0.725467
[26]	validation_0-auc:0.725748
[27]	validation_0-auc:0.725846
[28]	validation_0-auc:0.726167
[29]	validation_0-auc:0.726769
[30]	validation_0-auc

In [0]:
 xgb_grid.best_params_

{'max_depth': 9, 'min_child_weight': 1}

In [0]:
param1 = {
 'max_depth':[9,10],
 'min_child_weight':[1,2]
}
xgb_grid = GridSearchCV(xgb_best, param1, verbose=True, scoring = 'roc_auc', n_jobs=6, cv = 2)
xgb_grid.fit(X_train, y_train, eval_metric="auc", eval_set=eval_set, verbose=True, early_stopping_rounds=10)
y_pred = xgb_grid.predict_proba(X_test)[:,1]
print('AUC-ROC:', roc_auc_score(y_test, y_pred))
print(xgb_grid.best_params_)

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   6 out of   8 | elapsed:  7.1min remaining:  2.4min
[Parallel(n_jobs=6)]: Done   8 out of   8 | elapsed:  7.3min finished


[0]	validation_0-auc:0.697351
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.699445
[2]	validation_0-auc:0.711451
[3]	validation_0-auc:0.714769
[4]	validation_0-auc:0.716875
[5]	validation_0-auc:0.719173
[6]	validation_0-auc:0.720808
[7]	validation_0-auc:0.721199
[8]	validation_0-auc:0.721188
[9]	validation_0-auc:0.722347
[10]	validation_0-auc:0.722482
[11]	validation_0-auc:0.723072
[12]	validation_0-auc:0.723008
[13]	validation_0-auc:0.723456
[14]	validation_0-auc:0.724093
[15]	validation_0-auc:0.72444
[16]	validation_0-auc:0.724388
[17]	validation_0-auc:0.725497
[18]	validation_0-auc:0.725675
[19]	validation_0-auc:0.726254
[20]	validation_0-auc:0.72673
[21]	validation_0-auc:0.72686
[22]	validation_0-auc:0.727537
[23]	validation_0-auc:0.727807
[24]	validation_0-auc:0.728107
[25]	validation_0-auc:0.728511
[26]	validation_0-auc:0.728549
[27]	validation_0-auc:0.728823
[28]	validation_0-auc:0.728896
[29]	validation_0-auc:0.729112
[30]	validation_0-a

In [0]:
xgb_best = XGBClassifier(
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    learning_rate =0.1,
    n_estimators=50,
    max_depth=10,
    min_child_weight=2,
    scale_pos_weight=1)

param2 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
xgb_grid = GridSearchCV(xgb_best, param2, verbose=True, scoring = 'roc_auc', n_jobs=6, cv = 2)
xgb_grid.fit(X_train, y_train, eval_metric="auc", eval_set=eval_set, verbose=True, early_stopping_rounds=10)
y_pred = xgb_grid.predict_proba(X_test)[:,1]
print('AUC-ROC:', roc_auc_score(y_test, y_pred))
print(xgb_grid.best_params_)

Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:  9.6min remaining:    0.0s
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:  9.6min finished


[0]	validation_0-auc:0.697351
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.699445
[2]	validation_0-auc:0.711451
[3]	validation_0-auc:0.714769
[4]	validation_0-auc:0.716875
[5]	validation_0-auc:0.719173
[6]	validation_0-auc:0.720808
[7]	validation_0-auc:0.721199
[8]	validation_0-auc:0.721188
[9]	validation_0-auc:0.722347
[10]	validation_0-auc:0.722482
[11]	validation_0-auc:0.723072
[12]	validation_0-auc:0.723008
[13]	validation_0-auc:0.723456
[14]	validation_0-auc:0.724093
[15]	validation_0-auc:0.72444
[16]	validation_0-auc:0.724388
[17]	validation_0-auc:0.725497
[18]	validation_0-auc:0.725675
[19]	validation_0-auc:0.726254
[20]	validation_0-auc:0.72673
[21]	validation_0-auc:0.72686
[22]	validation_0-auc:0.727537
[23]	validation_0-auc:0.727807
[24]	validation_0-auc:0.728107
[25]	validation_0-auc:0.728511
[26]	validation_0-auc:0.728549
[27]	validation_0-auc:0.728823
[28]	validation_0-auc:0.728896
[29]	validation_0-auc:0.729112
[30]	validation_0-a

In [0]:
xgb_best = XGBClassifier(
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    learning_rate =0.1,
    n_estimators=50,
    max_depth=10,
    min_child_weight=2,
    scale_pos_weight=1)

param3 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
xgb_grid = GridSearchCV(xgb_best, param3, verbose=True, scoring = 'roc_auc', n_jobs=6, cv = 2)
xgb_grid.fit(X_train, y_train, eval_metric="auc", eval_set=eval_set, verbose=True, early_stopping_rounds=10)
y_pred = xgb_grid.predict_proba(X_test)[:,1]
print('AUC-ROC:', roc_auc_score(y_test, y_pred))
print(xgb_grid.best_params_)

Fitting 2 folds for each of 16 candidates, totalling 32 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  32 out of  32 | elapsed: 29.7min finished


[0]	validation_0-auc:0.699743
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.699862
[2]	validation_0-auc:0.712994
[3]	validation_0-auc:0.714229
[4]	validation_0-auc:0.716258
[5]	validation_0-auc:0.71675
[6]	validation_0-auc:0.719111
[7]	validation_0-auc:0.719961
[8]	validation_0-auc:0.720785
[9]	validation_0-auc:0.720336
[10]	validation_0-auc:0.720154
[11]	validation_0-auc:0.720402
[12]	validation_0-auc:0.72078
[13]	validation_0-auc:0.721653
[14]	validation_0-auc:0.722259
[15]	validation_0-auc:0.722371
[16]	validation_0-auc:0.72396
[17]	validation_0-auc:0.724783
[18]	validation_0-auc:0.725227
[19]	validation_0-auc:0.72605
[20]	validation_0-auc:0.726565
[21]	validation_0-auc:0.727363
[22]	validation_0-auc:0.727847
[23]	validation_0-auc:0.728592
[24]	validation_0-auc:0.728665
[25]	validation_0-auc:0.729017
[26]	validation_0-auc:0.729319
[27]	validation_0-auc:0.730397
[28]	validation_0-auc:0.730538
[29]	validation_0-auc:0.730842
[30]	validation_0-au

In [34]:
xgb_best = XGBClassifier(
    gamma=0,
    subsample=0.9,
    colsample_bytree=0.7,
    learning_rate =0.1,
    n_estimators=50,
    max_depth=10,
    min_child_weight=2,
    scale_pos_weight=1)

param4 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
xgb_grid = GridSearchCV(xgb_best, param4, verbose=True, scoring = 'roc_auc', n_jobs=6, cv = 2)
xgb_grid.fit(X_train, y_train, eval_metric="auc", eval_set=eval_set, verbose=True, early_stopping_rounds=10)
y_pred = xgb_grid.predict_proba(X_test)[:,1]
print('AUC-ROC:', roc_auc_score(y_test, y_pred))
print(xgb_grid.best_params_)

Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:  8.1min remaining:    0.0s
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:  8.1min finished


[0]	validation_0-auc:0.699742
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.69993
[2]	validation_0-auc:0.713067
[3]	validation_0-auc:0.714288
[4]	validation_0-auc:0.716309
[5]	validation_0-auc:0.716789
[6]	validation_0-auc:0.719148
[7]	validation_0-auc:0.719994
[8]	validation_0-auc:0.720812
[9]	validation_0-auc:0.720359
[10]	validation_0-auc:0.720175
[11]	validation_0-auc:0.72042
[12]	validation_0-auc:0.720797
[13]	validation_0-auc:0.721669
[14]	validation_0-auc:0.722274
[15]	validation_0-auc:0.722385
[16]	validation_0-auc:0.723975
[17]	validation_0-auc:0.724796
[18]	validation_0-auc:0.725241
[19]	validation_0-auc:0.726064
[20]	validation_0-auc:0.726577
[21]	validation_0-auc:0.727375
[22]	validation_0-auc:0.727859
[23]	validation_0-auc:0.728604
[24]	validation_0-auc:0.728677
[25]	validation_0-auc:0.729029
[26]	validation_0-auc:0.729331
[27]	validation_0-auc:0.730409
[28]	validation_0-auc:0.73055
[29]	validation_0-auc:0.730853
[30]	validation_0-a

In [35]:
xgb_best = XGBClassifier(
    gamma=0,
    subsample=0.9,
    colsample_bytree=0.7,
    learning_rate =0.1,
    n_estimators=50,
    max_depth=10,
    min_child_weight=2,
    scale_pos_weight=1)

param4 = {
 'reg_lambda':[1e-5, 1e-2, 0.1, 1, 100]
}
xgb_grid = GridSearchCV(xgb_best, param4, verbose=True, scoring = 'roc_auc', n_jobs=6, cv = 2)
xgb_grid.fit(X_train, y_train, eval_metric="auc", eval_set=eval_set, verbose=True, early_stopping_rounds=10)
y_pred = xgb_grid.predict_proba(X_test)[:,1]
print('AUC-ROC:', roc_auc_score(y_test, y_pred))
print(xgb_grid.best_params_)

Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:  7.9min remaining:    0.0s
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:  7.9min finished


[0]	validation_0-auc:0.699743
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.699862
[2]	validation_0-auc:0.712994
[3]	validation_0-auc:0.714229
[4]	validation_0-auc:0.716258
[5]	validation_0-auc:0.71675
[6]	validation_0-auc:0.719111
[7]	validation_0-auc:0.719961
[8]	validation_0-auc:0.720785
[9]	validation_0-auc:0.720336
[10]	validation_0-auc:0.720154
[11]	validation_0-auc:0.720402
[12]	validation_0-auc:0.72078
[13]	validation_0-auc:0.721653
[14]	validation_0-auc:0.722259
[15]	validation_0-auc:0.722371
[16]	validation_0-auc:0.72396
[17]	validation_0-auc:0.724783
[18]	validation_0-auc:0.725227
[19]	validation_0-auc:0.72605
[20]	validation_0-auc:0.726565
[21]	validation_0-auc:0.727363
[22]	validation_0-auc:0.727847
[23]	validation_0-auc:0.728592
[24]	validation_0-auc:0.728665
[25]	validation_0-auc:0.729017
[26]	validation_0-auc:0.729319
[27]	validation_0-auc:0.730397
[28]	validation_0-auc:0.730538
[29]	validation_0-auc:0.730842
[30]	validation_0-au

In [44]:
xgb_best = XGBClassifier(
    gamma=0,
    subsample=0.9,
    colsample_bytree=0.7,
    learning_rate =0.12,
    n_estimators=1000,
    max_depth=10,
    min_child_weight=2,
    scale_pos_weight=1,
    reg_lambda=1)

xgb_best.fit(X_train, y_train, eval_metric="auc", eval_set=eval_set, verbose=True, early_stopping_rounds=10)
y_pred = xgb_best.predict_proba(X_test)[:,1]
print('AUC-ROC:', roc_auc_score(y_test, y_pred))

[0]	validation_0-auc:0.699743
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.699443
[2]	validation_0-auc:0.712125
[3]	validation_0-auc:0.713943
[4]	validation_0-auc:0.716114
[5]	validation_0-auc:0.717298
[6]	validation_0-auc:0.719641
[7]	validation_0-auc:0.720753
[8]	validation_0-auc:0.721499
[9]	validation_0-auc:0.721767
[10]	validation_0-auc:0.72271
[11]	validation_0-auc:0.72285
[12]	validation_0-auc:0.723409
[13]	validation_0-auc:0.724419
[14]	validation_0-auc:0.724913
[15]	validation_0-auc:0.724863
[16]	validation_0-auc:0.726521
[17]	validation_0-auc:0.727427
[18]	validation_0-auc:0.727437
[19]	validation_0-auc:0.728456
[20]	validation_0-auc:0.728844
[21]	validation_0-auc:0.729621
[22]	validation_0-auc:0.730096
[23]	validation_0-auc:0.730276
[24]	validation_0-auc:0.730863
[25]	validation_0-auc:0.73118
[26]	validation_0-auc:0.731545
[27]	validation_0-auc:0.732045
[28]	validation_0-auc:0.732203
[29]	validation_0-auc:0.732496
[30]	validation_0-a

In [65]:
# Обучим еще классификатор для стекинга
X = df.copy()
y = X.loc[:,'dep_delayed_15min'] = X['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X = X.drop(columns=['dep_delayed_15min'])
X = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

rf = RandomForestClassifier(max_depth=9, n_estimators=120, min_samples_split=3)
rf.fit(X_train, y_train)
y_pred = rf.predict_proba(X_test)[:,1]
print('AUC-ROC:', roc_auc_score(y_test, y_pred))

AUC-ROC: 0.7041816587681633


In [70]:
# Блендинг

estimators = [
    ('rf', rf),
    ('xgb', xgb_best)
]
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3
)
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)[:,1]
print('AUC-ROC:', roc_auc_score(y_test, y_pred))

AUC-ROC: 0.7546289853062497
